This is a starter notebook for the project, you'll have to import the libraries you'll need, you can find a list of the ones available in this workspace in the requirements.txt file in this workspace. 

In [1]:
!pip install -r requirements.txt

Defaulting to user installation because normal site-packages is not writeable
  Using cached sentence_transformers-2.2.0-py3-none-any.whl
  Using cached huggingface_hub-0.14.1-py3-none-any.whl (224 kB)


  Attempting uninstall: huggingface-hub
    Found existing installation: huggingface-hub 0.15.1
    Uninstalling huggingface-hub-0.15.1:
      Successfully uninstalled huggingface-hub-0.15.1
  Consider adding this directory to PATH or, if you prefer to suppress this warning, use --no-warn-script-location.
  Attempting uninstall: sentence-transformers
    Found existing installation: sentence-transformers 2.2.2
    Uninstalling sentence-transformers-2.2.2:
      Successfully uninstalled sentence-transformers-2.2.2


In [2]:
# Dependency Updates
# Ensure dependencies are updated to avoid compatibility issues
!pip install --upgrade sentence-transformers==2.2.2 huggingface-hub==0.15.1

Defaulting to user installation because normal site-packages is not writeable
  Using cached sentence_transformers-2.2.2-py3-none-any.whl
  Using cached huggingface_hub-0.15.1-py3-none-any.whl (236 kB)
  Attempting uninstall: huggingface-hub
    Found existing installation: huggingface-hub 0.14.1
    Uninstalling huggingface-hub-0.14.1:
      Successfully uninstalled huggingface-hub-0.14.1
  Consider adding this directory to PATH or, if you prefer to suppress this warning, use --no-warn-script-location.
  Attempting uninstall: sentence-transformers
    Found existing installation: sentence-transformers 2.2.0
    Uninstalling sentence-transformers-2.2.0:
      Successfully uninstalled sentence-transformers-2.2.0


In [3]:
!pip show sentence-transformers huggingface-hub

Name: sentence-transformers
Version: 2.2.2
Summary: Multilingual text embeddings
Home-page: https://github.com/UKPLab/sentence-transformers
Author: Nils Reimers
Author-email: info@nils-reimers.de
License: Apache License 2.0
Location: /home/student/.local/lib/python3.10/site-packages
Requires: huggingface-hub, nltk, numpy, scikit-learn, scipy, sentencepiece, torch, torchvision, tqdm, transformers
Required-by: 
---
Name: huggingface-hub
Version: 0.15.1
Summary: Client library to download and publish models, datasets and other repos on the huggingface.co hub
Home-page: https://github.com/huggingface/huggingface_hub
Author: Hugging Face, Inc.
Author-email: julien@huggingface.co
License: Apache
Location: /home/student/.local/lib/python3.10/site-packages
Requires: filelock, fsspec, packaging, pyyaml, requests, tqdm, typing-extensions
Required-by: sentence-transformers, transformers


In [4]:
!pip show langchain openai chromadb

Name: langchain
Version: 0.0.305
Summary: Building applications with LLMs through composability
Home-page: https://github.com/langchain-ai/langchain
Author: 
Author-email: 
License: MIT
Location: /opt/conda/lib/python3.10/site-packages
Requires: aiohttp, anyio, async-timeout, dataclasses-json, jsonpatch, langsmith, numexpr, numpy, pydantic, PyYAML, requests, SQLAlchemy, tenacity
Required-by: 
---
Name: openai
Version: 0.28.1
Summary: Python client library for the OpenAI API
Home-page: https://github.com/openai/openai-python
Author: OpenAI
Author-email: support@openai.com
License: 
Location: /opt/conda/lib/python3.10/site-packages
Requires: aiohttp, requests, tqdm
Required-by: 
---
Name: chromadb
Version: 0.4.15
Summary: Chroma.
Home-page: 
Author: 
Author-email: Jeff Huber <jeff@trychroma.com>, Anton Troynikov <anton@trychroma.com>
License: 
Location: /home/student/.local/lib/python3.10/site-packages
Requires: bcrypt, chroma-hnswlib, fastapi, grpcio, importlib-resources, kubernetes, nu

In [5]:
# Verify dependency installation
import pkg_resources

def check_dependencies():
    required_packages = {
        "sentence-transformers": "2.2.2",
        "huggingface-hub": "0.15.1"
    }
    for package, version in required_packages.items():
        try:
            pkg_resources.require(f"{package}>={version}")
            print(f"{package} version is compatible.")
        except Exception as e:
            print(f"Issue with {package}: {e}")

check_dependencies()

sentence-transformers version is compatible.
huggingface-hub version is compatible.


In [6]:
# Import required modules
import traceback  # Handles error stack trace logging
import numpy as np
import chromadb  # Import ChromaDB client
from chromadb.config import Settings
import os
import logging

# Set up logging
logger = logging.getLogger("HomeMatch")
logging.basicConfig(level=logging.DEBUG)  # Enable DEBUG logging for detailed insights

# Mock embedding function to simulate the generation of embeddings
def mock_generate_embedding(text):
    embedding_size = 300  # Ensure consistent embedding size
    return np.random.rand(embedding_size).astype(np.float32)  # Random float embeddings for testing

# Step 7: Test the Application with ChromaDB
def test_application_chromadb():
    try:
        logger.info("===== Starting Application Test (ChromaDB) =====")

        # Reset ChromaDB directory for fresh start
        db_directory = "./chroma_db"
        if os.path.exists(db_directory):
            for file in os.listdir(db_directory):
                os.remove(os.path.join(db_directory, file))
        os.makedirs(db_directory, exist_ok=True)

        # Create ChromaDB client
        client = chromadb.Client(Settings(persist_directory=db_directory))
        collection_name = "real_estate_listings"

        # Create or open collection
        if collection_name not in [col.name for col in client.list_collections()]:
            collection = client.create_collection(name=collection_name)
            logger.info(f"Collection '{collection_name}' created successfully.")
        else:
            collection = client.get_collection(name=collection_name)
            logger.info(f"Collection '{collection_name}' already exists.")

        # Buyer preferences
        buyer_preferences = {
            "Property Type": "House",
            "Number of Bedrooms": 3,
            "Neighborhood": "Quiet",
            "Feature": "Backyard"
        }

        # Test listings
        test_listings = [
            {"id": "listing_1", "description": "Cozy 3-bedroom house", "price": 250000},
            {"id": "listing_2", "description": "Modern apartment in the city", "price": 400000},
            {"id": "listing_3", "description": "Luxury villa with pool", "price": 800000}
        ]

        # Store listings in ChromaDB
        for listing in test_listings:
            embedding = mock_generate_embedding(listing["description"])
            collection.add(
                documents=[listing["description"]],
                embeddings=[embedding.tolist()],
                ids=[listing["id"]],
                metadatas=[{"price": listing["price"]}]
            )
            logger.info(f"Stored listing {listing['id']} successfully.")

        # Semantic search
        query = f"{buyer_preferences['Property Type']}, {buyer_preferences['Number of Bedrooms']} bedrooms, {buyer_preferences['Neighborhood']}, {buyer_preferences['Feature']}"
        query_embedding = mock_generate_embedding(query)

        # Perform vector search with ChromaDB
        logger.info("Performing search with ChromaDB...")
        search_results = collection.query(
            query_embeddings=[query_embedding.tolist()],
            n_results=5,
            include=["documents", "metadatas"]
        )

        # Inspect search results
        if not search_results["documents"]:
            logger.warning("No listings matched the buyer's preferences.")
            return []

        logger.info(f"Search results: {search_results['documents']}")
        return search_results["documents"]

    except Exception as e:
        logger.error(f"Test failed with unexpected error: {e}")
        traceback.print_exc()
        return []

# Run the test
logger.info("===== Running Application Test (ChromaDB) =====")
test_results = test_application_chromadb()
logger.info(f"Test results: {test_results}")


INFO:HomeMatch:===== Running Application Test (ChromaDB) =====
INFO:HomeMatch:===== Starting Application Test (ChromaDB) =====
INFO:chromadb.telemetry.product.posthog:Anonymized telemetry enabled. See                     https://docs.trychroma.com/telemetry for more information.
DEBUG:chromadb.config:Starting component System
DEBUG:chromadb.config:Starting component Posthog
DEBUG:chromadb.config:Starting component OpenTelemetryClient
DEBUG:chromadb.config:Starting component SimpleAssignmentPolicy
DEBUG:chromadb.config:Starting component SqliteDB
DEBUG:chromadb.config:Starting component LocalSegmentManager
DEBUG:chromadb.config:Starting component SegmentAPI
INFO:HomeMatch:Collection 'real_estate_listings' created successfully.
DEBUG:chromadb.config:Starting component LocalHnswSegment
INFO:HomeMatch:Stored listing listing_1 successfully.
INFO:HomeMatch:Stored listing listing_2 successfully.
INFO:HomeMatch:Stored listing listing_3 successfully.
INFO:HomeMatch:Performing search with Chroma

In [7]:

logger.info("Finished...")

INFO:HomeMatch:Finished...


In [8]:
import os
import openai

openai.api_key = ""
os.environ["OPENAI_API_KEY"] = os.getenv("OPENAI_API_KEY", openai.api_key)


try:
    response = openai.ChatCompletion.create(
        model="gpt-3.5-turbo",
        messages=[
            {"role": "system", "content": "You are a helpful assistant."},
            {"role": "user", "content": "Say hello!"}
        ]
    )
    print("API Test Successful:", response['choices'][0]['message']['content'].strip())
except openai.error.OpenAIError as e:
    print("API Test Failed:", str(e))

DEBUG:openai:message='Request to OpenAI API' method=post path=https://api.openai.com/v1/chat/completions
DEBUG:openai:api_version=None data='{"model": "gpt-3.5-turbo", "messages": [{"role": "system", "content": "You are a helpful assistant."}, {"role": "user", "content": "Say hello!"}]}' message='Post details'
DEBUG:urllib3.util.retry:Converted retries value: 2 -> Retry(total=2, connect=None, read=None, redirect=None, status=None)
DEBUG:urllib3.connectionpool:Starting new HTTPS connection (1): api.openai.com:443
DEBUG:urllib3.connectionpool:Starting new HTTPS connection (1): us-api.i.posthog.com:443
DEBUG:urllib3.connectionpool:https://api.openai.com:443 "POST /v1/chat/completions HTTP/1.1" 200 None
DEBUG:openai:message='OpenAI API response' path=https://api.openai.com/v1/chat/completions processing_ms=289 request_id=req_ffb780a75aea3eb4571472611ec71ed3 response_code=200


API Test Successful: Hello! How can I assist you today?


In [9]:

import openai

# Ensure your OpenAI API key is set

try:
    response = openai.Embedding.create(
        input="Test input text",
        model="text-embedding-ada-002"
    )
    embedding = response['data'][0]['embedding']
    print("Embedding generated successfully:", embedding[:5])
except openai.error.OpenAIError as e:
    print("Embedding generation failed:", str(e))



DEBUG:openai:message='Request to OpenAI API' method=post path=https://api.openai.com/v1/embeddings
DEBUG:openai:api_version=None data='{"input": "Test input text", "model": "text-embedding-ada-002", "encoding_format": "base64"}' message='Post details'
DEBUG:urllib3.connectionpool:https://us-api.i.posthog.com:443 "POST /batch/ HTTP/1.1" 200 15
DEBUG:urllib3.connectionpool:https://api.openai.com:443 "POST /v1/embeddings HTTP/1.1" 200 None
DEBUG:openai:message='OpenAI API response' path=https://api.openai.com/v1/embeddings processing_ms=80 request_id=req_1ab67227a91220f02d076835bb3c8b64 response_code=200


Embedding generated successfully: [-0.007740851026028395, 0.002270322060212493, -0.011116507463157177, 0.024282269179821014, -0.013537716120481491]


Step 1: Setting Up the Python Application

Initialize a Python Project: Create a new Python project, setting up a virtual environment and installing necessary packages like LangChain, a suitable LLM library (e.g., OpenAI's GPT), and a vector database package compatible with Python (e.g., ChromaDB or LanceDB). If you don't wish to create your files from scratch, starter files are available in the workspace on the next page as an application skeleton.

In [10]:
# Step 1: Setting Up the Application and ChromaDB
import os
import logging
import traceback
from time import time
from chromadb import Client
from chromadb.config import Settings
import shutil  # To clear stale persistence directories if needed
from chromadb.utils import embedding_functions

# Logging setup
logger = logging.getLogger("HomeMatch-Step1")
logging.basicConfig(level=logging.DEBUG)

# Global Variables
CHROMADB_CLIENT = None
COLLECTION_CACHE = None
EMBEDDING_MODEL = "text-embedding-ada-002"  # OpenAI embedding model
EMBEDDING_DIMENSION = 1536  # Dimensionality for the embedding model
persist_dir = "./chroma_db"  # Directory for ChromaDB persistence

# Environment Variables Setup
api_key = os.getenv("OPENAI_API_KEY")
base_url = os.getenv("OPENAI_API_BASE", "https://api.openai.com/v1")  # Default base URL if missing

if api_key:
    os.environ["OPENAI_API_KEY"] = api_key
else:
    logger.error("OPENAI_API_KEY is missing! Please set your API key.")

os.environ["OPENAI_API_BASE"] = base_url  # Ensure it is set for the environment
os.environ["CHROMA_TELEMETRY_DISABLED"] = "1"  # Optional: Disable ChromaDB telemetry

# Log the environment variables to verify values
logger.info(f"Embedding model set to: {EMBEDDING_MODEL}")
logger.info(f"OpenAI API key set: {'Yes' if api_key else 'No'}")
logger.info(f"OpenAI API base URL: {os.environ['OPENAI_API_BASE']}")
logger.info("Environment variables set up.")

# Check the persistence directory
if os.path.exists(persist_dir):
    logger.info(f"ChromaDB persistence directory exists. Contents: {os.listdir(persist_dir)}")
else:
    logger.warning(f"Persistence directory '{persist_dir}' does not exist. It will be created on first use.")

# Optionally clear persistence directory to avoid stale data
def clear_persistence():
    if os.path.exists(persist_dir):
        shutil.rmtree(persist_dir)
        logger.info(f"Deleted stale persistence directory '{persist_dir}'. Starting fresh...")

# System Checks
def check_environment_variables():
    """Checks for required environment variables and sets defaults if missing."""
    required_vars = {
        "OPENAI_API_KEY": None,  # Must be set
        "OPENAI_API_BASE": "https://api.openai.com/v1"  # Default value
    }
    
    missing_vars = []
    for var, default_value in required_vars.items():
        if not os.environ.get(var):
            if default_value:
                os.environ[var] = default_value  # Set default value if missing
                logger.warning(f"{var} not set. Using default value: {default_value}")
            else:
                missing_vars.append(var)
    
    if missing_vars:
        logger.error(f"Missing critical environment variables: {missing_vars}")
        return False
    logger.info("All required environment variables are set.")
    return True

def check_api_quota():
    """Checks OpenAI API quota with a robust retry mechanism."""
    from tenacity import retry, stop_after_attempt, wait_exponential

    @retry(stop=stop_after_attempt(3), wait=wait_exponential(multiplier=1, max=10))
    def test_quota():
        logger.info("Testing OpenAI API connectivity...")
        try:
            # Test embedding creation with dummy text
            response = openai.Embedding.create(input=["test"], model=EMBEDDING_MODEL)
            logger.info("OpenAI API connectivity and quota confirmed.")
            return True
        except Exception as e:
            logger.error(f"Error during API quota test: {e}")
            raise

    try:
        return test_quota()
    except Exception as e:
        logger.error(f"API quota check failed after retries: {e}")
        return False

# Initialize ChromaDB Client
def initialize_chromadb_client():
    """Initialize the ChromaDB client with persistence settings."""
    global CHROMADB_CLIENT
    if CHROMADB_CLIENT is None:
        try:
            if not os.path.exists(persist_dir):
                os.makedirs(persist_dir)
                logger.info(f"Created persistence directory: {persist_dir}")
            settings = Settings(persist_directory="./chroma_db")
            CHROMADB_CLIENT = Client(settings)  # Ensure this line is indented with 4 spaces consistently
        except Exception as e:
            logger.error(f"Failed to initialize ChromaDB client: {e}")
            logger.debug(f"Traceback: {traceback.format_exc()}")
            raise
    return CHROMADB_CLIENT


def validate_collection_after_store(collection):
    try:
        sample_data = collection.get(include=["embeddings"])
        for idx, embedding in enumerate(sample_data.get("embeddings", [])):
            if len(embedding) != EMBEDDING_DIMENSION:
                logger.error(f"Embedding mismatch in stored document {idx}. Expected: {EMBEDDING_DIMENSION}, Got: {len(embedding)}")
    except Exception as e:
        logger.error(f"Error validating collection embeddings post-store: {e}")



def generate_embeddings(text):
    """
    Generate embeddings using OpenAI's embedding model.

    Args:
        text (str): The input text.

    Returns:
        list: The embedding vector.
    """
    try:
        logger.info(f"Generating embeddings for text: '{text[:50]}...'")
        response = openai.Embedding.create(input=[text], model=EMBEDDING_MODEL)
        embedding = response["data"][0]["embedding"]
        if len(embedding) != EMBEDDING_DIMENSION:
            raise ValueError(f"Generated embedding dimension mismatch. Expected: {EMBEDDING_DIMENSION}, "
                             f"but got: {len(embedding)}.")
        logger.debug(f"Generated embedding (First 10 values): {embedding[:10]}...")
        return embedding
    except Exception as e:
        logger.error(f"Error generating embeddings: {e}")
        raise

def validate_collection_dimensionality(collection):
    """
    Validate the embedding dimensionality of the ChromaDB collection.

    Args:
        collection (Collection): The ChromaDB collection to validate.

    Returns:
        bool: True if the collection has the correct dimensionality, False otherwise.
    """
    try:
        sample_docs = collection.get(include=["embeddings"])
        if sample_docs.get("embeddings") and len(sample_docs["embeddings"][0]) != EMBEDDING_DIMENSION:
            logger.warning(f"Collection embedding dimension mismatch. Expected: {EMBEDDING_DIMENSION}, "
                           f"but found: {len(sample_docs['embeddings'][0])}.")
            return False
        logger.info("Collection embedding dimension validated successfully.")
        return True
    except Exception as e:
        logger.error(f"Error validating collection dimensionality: {e}")
        return False
    
def dump_full_collection(collection):
    """Dump all documents and metadata from the collection."""
    try:
        all_data = collection.get(include=["documents", "metadatas"])
        logger.info(f"Full contents of collection '{collection.name}':\n{all_data}")
    except Exception as e:
        logger.error(f"Error dumping collection contents: {e}")

def log_collection_info(collection):
    """
    Logs information about the collection's contents.
    
    Args:
        collection (Collection): The ChromaDB collection to log.
    """
    try:
        collection_size = len(collection.get(include=["documents"])['documents'])
        sample_data = collection.get(limit=min(3, collection_size), include=["documents", "metadatas"])  # Changed 'n_results' to 'limit'
        logger.info(f"[log_collection_info] Collection '{collection.name}' contains {collection_size} documents.")
        for idx, (doc, meta) in enumerate(zip(sample_data['documents'], sample_data['metadatas'])):
            logger.info(f"[log_collection_info] Sample {idx + 1}: Document = {doc[:100]}..., Metadata = {meta}")
    except Exception as e:
        logger.error(f"[log_collection_info] Error logging collection info: {e}")
        logger.debug(f"[log_collection_info] Traceback: {traceback.format_exc()}")


# Get or Create Collection
def get_collection(collection_name="real_estate_listings"):
    """Retrieve or create a ChromaDB collection."""
    global COLLECTION_CACHE
    client = initialize_chromadb_client()
    # Define the OpenAI embedding function explicitly
    openai_ef = embedding_functions.OpenAIEmbeddingFunction(
        api_key=os.getenv("OPENAI_API_KEY"),
        model_name="text-embedding-ada-002"
    )

    try:
        logger.info(f"Validating or retrieving collection '{collection_name}'...")
        existing_collections = [col.name for col in client.list_collections()]
        if collection_name in existing_collections:
            COLLECTION_CACHE = client.get_collection(name=collection_name, embedding_function=openai_ef)
            if not validate_collection_dimensionality(COLLECTION_CACHE):
                logger.warning(f"Recreating collection '{collection_name}' due to dimensionality mismatch.")
                client.delete_collection(name=collection_name)
                COLLECTION_CACHE = client.create_collection(name=collection_name, embedding_function=openai_ef)
                logger.info(f"Recreated collection '{collection_name}' with correct dimensions.")
        else:
            COLLECTION_CACHE = client.create_collection(name=collection_name)
            logger.info(f"Created new collection '{collection_name}'.")
    except Exception as e:
        logger.error(f"Failed to validate or retrieve collection '{collection_name}': {e}")
        logger.debug(f"Traceback: {traceback.format_exc()}")
        raise
    return COLLECTION_CACHE

# Run Setup Process
start_time = time()  # Track setup duration
logger.info("Running system checks...")

if not check_environment_variables():
    raise EnvironmentError("Critical environment variables are missing.")

if not check_api_quota():
    logger.warning("Insufficient API quota or model access. Functionality may be limited.")
else:
    logger.info("All system checks passed.")

# Initialize ChromaDB Client and Collection
try:
    CHROMADB_CLIENT = initialize_chromadb_client()
    COLLECTION_CACHE = get_collection(collection_name="real_estate_listings")
    logger.info("ChromaDB client and collection initialized successfully.")
except Exception as e:
    logger.error(f"Error during ChromaDB setup: {e}")
    raise

# Verify Collection Size
try:
    collection_size = len(COLLECTION_CACHE.get(include=["documents"])["documents"])
    logger.info(f"ChromaDB collection size: {collection_size} documents.")
except Exception as e:
    logger.error(f"Error inspecting ChromaDB collection: {e}")

logger.info(f"Step 1: Application setup complete. Total setup time: {time() - start_time:.2f} seconds.")

logger.info(f"Contents of './chroma_db' after restart: {os.listdir('./chroma_db') if os.path.exists('./chroma_db') else 'Directory missing'}")



INFO:HomeMatch-Step1:Embedding model set to: text-embedding-ada-002
INFO:HomeMatch-Step1:OpenAI API key set: Yes
INFO:HomeMatch-Step1:OpenAI API base URL: https://api.openai.com/v1
INFO:HomeMatch-Step1:Environment variables set up.
INFO:HomeMatch-Step1:ChromaDB persistence directory exists. Contents: []
INFO:HomeMatch-Step1:Running system checks...
INFO:HomeMatch-Step1:All required environment variables are set.
INFO:HomeMatch-Step1:Testing OpenAI API connectivity...
DEBUG:openai:message='Request to OpenAI API' method=post path=https://api.openai.com/v1/embeddings
DEBUG:openai:api_version=None data='{"input": ["test"], "model": "text-embedding-ada-002", "encoding_format": "base64"}' message='Post details'
DEBUG:urllib3.connectionpool:https://api.openai.com:443 "POST /v1/embeddings HTTP/1.1" 200 None
DEBUG:openai:message='OpenAI API response' path=https://api.openai.com/v1/embeddings processing_ms=49 request_id=req_18b9e5384a5725d707b99b276869739e response_code=200
INFO:HomeMatch-Step1:

In [11]:
import openai
import logging
from tenacity import retry, stop_after_attempt, wait_exponential

# Initialize Logging
logger = logging.getLogger("HomeMatch")
logging.basicConfig(level=logging.DEBUG)  # Set to DEBUG for detailed logs

# Define Budget and Token Tracker
TOTAL_BUDGET = 1_000_000  # Replace with your actual token budget
used_tokens = 0

def log_usage(tokens_used):
    """Log and track token usage."""
    global used_tokens
    used_tokens += tokens_used
    remaining_budget = TOTAL_BUDGET - used_tokens
    logger.info(f"Tokens Used: {tokens_used}, Total Used: {used_tokens}, Remaining Budget: {remaining_budget}")
    if remaining_budget <= 0:
        logger.error(f"Budget exceeded: Total Used: {used_tokens}, Remaining Budget: {remaining_budget}")
        raise RuntimeError("API Budget exceeded. No tokens remaining!")

@retry(stop=stop_after_attempt(3), wait=wait_exponential(multiplier=1, min=2, max=10))
def make_chat_request(messages):
    """Make a chat request to OpenAI."""
    try:
        logger.debug(f"Sending chat request with messages: {messages}")
        response = openai.ChatCompletion.create(
            model="gpt-3.5-turbo",
            messages=messages
        )
        if "usage" not in response or "total_tokens" not in response["usage"]:
            logger.error(f"Invalid response format: {response}")
            raise ValueError("Invalid API response: Missing usage information")
        tokens_used = response["usage"]["total_tokens"]
        log_usage(tokens_used)
        return response["choices"][0]["message"]["content"]
    except openai.error.OpenAIError as e:
        logger.error(f"OpenAI API error: {str(e)}")
        raise

def make_embedding_request(texts, batch_size=10):
    """Batch and request embeddings from OpenAI."""
    embeddings = []
    for i in range(0, len(texts), batch_size):
        batch = texts[i:i+batch_size]
        logger.debug(f"Processing batch {i//batch_size + 1}: {batch}")
        try:
            response = openai.Embedding.create(
                input=batch,
                model="text-embedding-ada-002"
            )
            if "usage" not in response or "data" not in response:
                logger.error(f"Invalid response format: {response}")
                raise ValueError("Invalid API response: Missing embedding data or usage")
            tokens_used = response["usage"]["total_tokens"]
            log_usage(tokens_used)
            embeddings.extend([item["embedding"] for item in response["data"]])
        except openai.error.OpenAIError as e:
            logger.error(f"Error in embedding request: {str(e)}")
            raise
    return embeddings

# Example Usage
messages = [
    {"role": "system", "content": "You are a helpful assistant."},
    {"role": "user", "content": "Provide insights on token usage."}
]
try:
    result = make_chat_request(messages)
    logger.info(f"Chat Result: {result}")

    texts = ["Sample text for embedding"] * 15  # Example input
    embeddings = make_embedding_request(texts, batch_size=5)
    logger.info(f"Generated {len(embeddings)} embeddings.")
except RuntimeError as e:
    logger.error(str(e))


DEBUG:HomeMatch:Sending chat request with messages: [{'role': 'system', 'content': 'You are a helpful assistant.'}, {'role': 'user', 'content': 'Provide insights on token usage.'}]
DEBUG:openai:message='Request to OpenAI API' method=post path=https://api.openai.com/v1/chat/completions
DEBUG:openai:api_version=None data='{"model": "gpt-3.5-turbo", "messages": [{"role": "system", "content": "You are a helpful assistant."}, {"role": "user", "content": "Provide insights on token usage."}]}' message='Post details'
DEBUG:urllib3.connectionpool:https://us-api.i.posthog.com:443 "POST /batch/ HTTP/1.1" 200 15
DEBUG:urllib3.connectionpool:https://api.openai.com:443 "POST /v1/chat/completions HTTP/1.1" 200 None
DEBUG:openai:message='OpenAI API response' path=https://api.openai.com/v1/chat/completions processing_ms=5325 request_id=req_e0a0f30dfc7eaf909a0487c2f2ca88ae response_code=200
INFO:HomeMatch:Tokens Used: 345, Total Used: 345, Remaining Budget: 999655
INFO:HomeMatch:Chat Result: Tokens are 

In [12]:
# Test the OpenAI API connection
# Step 1: Test the OpenAI API
@retry(stop=stop_after_attempt(3), wait=wait_exponential(multiplier=1, min=2, max=10))
def test_api_connection():
    import openai

    if "OPENAI_API_KEY" not in os.environ:
        logger.error("OPENAI_API_KEY is not set. Cannot test API connection.")
        return False

    openai.api_key = os.environ["OPENAI_API_KEY"]
    try:
        logger.debug("Sending test request to OpenAI API...")
        response = openai.ChatCompletion.create(
            model="gpt-3.5-turbo",
            messages=[
                {"role": "system", "content": "You are a helpful assistant."},
                {"role": "user", "content": "Say hello!"}
            ]
        )
        if "choices" not in response or not response.choices:
            logger.error("Invalid API response format: %s", response)
            return False
        message_content = response.choices[0].message.content.strip()
        logger.info("API Test Successful: %s", message_content)
        return True
    except openai.error.OpenAIError as e:
        logger.error("API Test Failed: [%s] %s", e.__class__.__name__, str(e))
        return False
    except Exception as general_error:
        logger.error("Unexpected error during API test: [%s] %s", general_error.__class__.__name__, str(general_error))
        return False

# Run API test
if test_api_connection():
    logger.info("OpenAI API connection is operational.")
else:
    logger.error("OpenAI API connection test failed.")


DEBUG:HomeMatch:Sending test request to OpenAI API...
DEBUG:openai:message='Request to OpenAI API' method=post path=https://api.openai.com/v1/chat/completions
DEBUG:openai:api_version=None data='{"model": "gpt-3.5-turbo", "messages": [{"role": "system", "content": "You are a helpful assistant."}, {"role": "user", "content": "Say hello!"}]}' message='Post details'
DEBUG:urllib3.connectionpool:https://api.openai.com:443 "POST /v1/chat/completions HTTP/1.1" 200 None
DEBUG:openai:message='OpenAI API response' path=https://api.openai.com/v1/chat/completions processing_ms=323 request_id=req_df3e397098fca536198f6fccfb44c0b7 response_code=200
INFO:HomeMatch:API Test Successful: Hello! How can I help you today?
INFO:HomeMatch:OpenAI API connection is operational.


Step 2: Generating Real Estate Listings

Generate real estate listings using a Large Language Model. Generate at least 10 listings This can involve creating prompts for the LLM to produce descriptions of various properties. An example of a listing might be:
Neighborhood: Green Oaks
Price: $800,000
Bedrooms: 3
Bathrooms: 2
House Size: 2,000 sqft

Description: Welcome to this eco-friendly oasis nestled in the heart of Green Oaks. This charming 3-bedroom, 2-bathroom home boasts energy-efficient features such as solar panels and a well-insulated structure. Natural light floods the living spaces, highlighting the beautiful hardwood floors and eco-conscious finishes. The open-concept kitchen and dining area lead to a spacious backyard with a vegetable garden, perfect for the eco-conscious family. Embrace sustainable living without compromising on style in this Green Oaks gem.

Neighborhood Description: Green Oaks is a close-knit, environmentally-conscious community with access to organic grocery stores, community gardens, and bike paths. Take a stroll through the nearby Green Oaks Park or grab a cup of coffee at the cozy Green Bean Cafe. With easy access to public transportation and bike lanes, commuting is a breeze.
You'll use these listings to populate the database for testing and development of "HomeMatch".

In [13]:
# Step 2: Generating Real Estate Listings with an LLM
import openai
import logging
import random
import os
import json

# Set up logging
logger = logging.getLogger("HomeMatch-Step2")
logging.basicConfig(level=logging.DEBUG)

# File path to save and read listings
listings_file_path = "listings.txt"

# Define expected embedding dimension
EMBEDDING_DIMENSION = 1536

# Function to save listings to a file
def save_listings_to_file(listings, file_path):
    """
    Save listings to a text file.

    Args:
        listings (list): List of real estate listings.
        file_path (str): Path to the file where listings are saved.
    """
    logger.info("save_listings_to_file: Saving listings to file...")
    with open(file_path, "w") as file:
        for i, listing in enumerate(listings, start=1):
#           logger.info(f"[save_listings_to_file] Saving Listing {i}: {json.dumps(listing, indent=4)}")
            file.write(f"Listing {i}:\n")
            file.write(json.dumps(listing, indent=4))
            file.write("\n\n")
    logger.info(f"[save_listings_to_file] Listings saved to {file_path}")

# Function to read listings from a file
def read_listings_from_file(file_path):
    """
    Read listings from a text file if it exists.

    Args:
        file_path (str): Path to the file.

    Returns:
        list: A list of dictionaries containing real estate listings.
    """
    logger.info("[read_listings_from_file] Reading listings from file...")
    if os.path.exists(file_path):
        with open(file_path, "r") as file:
            content = file.read().strip()
            if not content:
                logger.warning(f"[read_listings_from_file] File {file_path} is empty. Deleting the file and regenerating listings.")
                os.remove(file_path)
                return []
            try:
                listings = content.split("\n\n")
                parsed_listings = [json.loads(listing) for listing in listings if listing.strip()]
                return parsed_listings
            except json.JSONDecodeError as e:
                logger.error(f"[read_listings_from_file] Error parsing JSON from {file_path}: {e}. Deleting corrupted file.")
                os.remove(file_path)
                return []
    logger.warning(f"[read_listings_from_file] File {file_path} not found.")
    return []

# Function to validate generated listings
def validate_listings(listings):
    """
    Validate the structure and embedding compatibility of generated listings.

    Args:
        listings (list): List of generated listings.

    Returns:
        list: Validated listings.
    """
    logger.info("[validate_listings] Validating listings...")
    valid_listings = []
    for idx, listing in enumerate(listings, start=1):
        logger.info(f"[validate_listings] Validating Listing {idx}: {json.dumps(listing, indent=4)}")
        # Ensure all required fields are present
        if len(listing["description"]) == 0:
            logger.warning(f"[validate_listings] Listing {idx} has an empty description. Skipping...")
            continue

        valid_listings.append(listing)

    logger.info(f"[validate_listings] Validated {len(valid_listings)} out of {len(listings)} listings.")
    return valid_listings

# Function to generate real estate listings using GPT
def generate_real_estate_listings(num_listings=10):
    """
    Generate real estate listings using an LLM (OpenAI GPT-3.5 Turbo).

    Args:
        num_listings (int): Number of listings to generate.

    Returns:
        list: A list of dictionaries containing real estate listings.
    """
    openai.api_key = os.getenv("OPENAI_API_KEY")

    if not openai.api_key:
        logger.error("[generate_real_estate_listings] OpenAI API key is missing. Set the OPENAI_API_KEY environment variable.")
        raise RuntimeError("[generate_real_estate_listings] OpenAI API key is missing.")

    logger.info(f"[generate_real_estate_listings] Generating {num_listings} real estate listings using OpenAI GPT...")

    listings = []  # List to store generated listings

    prompt_template = (
        "You are a real estate expert. Generate a realistic real estate listing with the following format:\n"
        "Neighborhood: <neighborhood name>\n"
        "Price: $<price>\n"
        "Bedrooms: <number of bedrooms>\n"
        "Bathrooms: <number of bathrooms>\n"
        "House Size: <square footage>\n"
        "Description: <detailed description of the property, including features>\n"
        "Neighborhood Description: <brief description of the neighborhood and amenities nearby>"
    )

    neighborhoods = ["Green Oaks", "Maple Heights", "Sunnydale", "Brookside", "Willow Creek"]
    price_ranges = [(250000, 500000), (500000, 750000), (750000, 1000000)]

    for _ in range(num_listings):
        neighborhood = random.choice(neighborhoods)
        price_range = random.choice(price_ranges)
        price = random.randint(*price_range)

        prompt = f"{prompt_template}\nNeighborhood: {neighborhood}\nPrice: ${price}\n"

        try:
            response = openai.ChatCompletion.create(
                model="gpt-3.5-turbo",
                messages=[
                    {"role": "system", "content": "You are a helpful assistant."},
                    {"role": "user", "content": prompt}
                ],
                max_tokens=300,
                temperature=0.7
            )
            listing_text = response["choices"][0]["message"]["content"].strip()
            logger.debug(f"[generate_real_estate_listings] Generated Listing Text: {listing_text}")

            listing = {
                "id": f"listing_{len(listings) + 1}",
                "neighborhood": neighborhood,
                "price": price,
                "description": listing_text
            }
            listings.append(listing)
            logger.info(f"[generate_real_estate_listings] Generated Listing {len(listings)}: {json.dumps(listing, indent=4)}")

        except Exception as e:
            logger.error(f"[generate_real_estate_listings] Error generating listing: {e}")
            continue

    logger.info(f"[generate_real_estate_listings] Successfully generated {len(listings)} listings.")
    listings = validate_listings(listings)

    return listings



Step 3: Storing Listings in a Vector Database

Vector Database Setup: Initialize and configure ChromaDB or a similar vector database to store real estate listings.
Generating and Storing Embeddings: Convert the LLM-generated listings into suitable embeddings that capture the semantic content of each listing, and store these embeddings in the vector database.

In [14]:
# Step 3: Storing Listings in ChromaDB
import logging
import traceback
import os

# Logging setup
logger = logging.getLogger("HomeMatch-Step3")
logging.basicConfig(level=logging.DEBUG)

def clean_metadata(metadata):
    """
    Cleans metadata values to ensure they are valid types for ChromaDB.
    Args:
        metadata (dict): The metadata dictionary to clean.
    Returns:
        dict: Cleaned metadata with no None values.
    """
    cleaned_metadata = {}
    for key, value in metadata.items():
        if value is None:
            # Provide default values for None based on expected data type
            if key == "bedrooms":  # Example numeric field
                cleaned_metadata[key] = 0
            elif key == "price":  # Example numeric field
                cleaned_metadata[key] = 0.0
            else:
                cleaned_metadata[key] = "Unknown"
        else:
            cleaned_metadata[key] = value
    return cleaned_metadata

def store_listings_in_chromadb(listings):
    """
    Store generated real estate listings in ChromaDB using the client and collection.

    Args:
        listings (list): List of real estate listings to store.
    """
    if not listings:
        logger.warning("[store_listings_in_chromadb] No listings to store.")
        return

    logger.info(f"[store_listings_in_chromadb] Received {len(listings)} listings for storage.")
    logger.info(f"[store_listings_in_chromadb] Sample listings data:\n{listings[:3]}")  # Log sample listings for verification

    try:
        # Retrieve or validate the collection
        collection = get_collection(collection_name="real_estate_listings")
        logger.info(f"[store_listings_in_chromadb] Embedding function in use: {collection._embedding_function}")
        logger.info(f"[store_listings_in_chromadb] Using collection '{collection.name}' to store listings.")

        # Validate collection dimensionality before storing
        if not validate_collection_dimensionality(collection):
            logger.warning("[store_listings_in_chromadb] Collection dimensionality mismatch detected. Recreating collection...")
            collection = get_collection(collection_name="real_estate_listings")
            logger.info(f"[store_listings_in_chromadb] Recreated collection '{collection.name}' successfully.")

        initial_count = len(collection.get(include=["documents"])['documents'])
        logger.info(f"[store_listings_in_chromadb] Initial collection size: {initial_count} documents.")

        for idx, listing in enumerate(listings, start=1):
            logger.info(f"[store_listings_in_chromadb] Processing Listing {idx}: {listing}")

            # Validate the structure of the listing
            required_fields = ["id", "neighborhood", "price", "description"]
            if not all(key in listing for key in required_fields):
                logger.warning(f"[store_listings_in_chromadb] Listing {idx} is missing required fields {required_fields}. Skipping...")
                continue

            logger.info(f"[store_listings_in_chromadb] Storing listing {idx}: ID = {listing['id']}, "
                        f"Neighborhood = {listing['neighborhood']}, Price = ${listing['price']}")

            # Generate embeddings for the description
            try:
                logger.debug(f"[store_listings_in_chromadb] Generating embedding for listing {idx} description: '{listing['description'][:50]}...'")
                embedding = generate_embeddings(listing["description"])
                logger.info(f"[store_listings_in_chromadb] Generated embedding for listing {idx} (length): {len(embedding)}")

                # Validate the embedding dimensionality
                if len(embedding) != EMBEDDING_DIMENSION:
                    logger.error(f"[store_listings_in_chromadb] Embedding dimension mismatch for listing {idx}. "
                                 f"Expected {EMBEDDING_DIMENSION}, got {len(embedding)}.")
                    continue
            except Exception as embedding_error:
                logger.error(f"[store_listings_in_chromadb] Error generating embedding for listing {idx}: {embedding_error}")
                continue

            # Store the listing in the collection
            try:
                metadata = {
                    "neighborhood": listing.get("neighborhood"),
                    "price": listing.get("price"),
                    "id": listing.get("id"),
                    "bedrooms": listing.get("bedrooms"),
                    "property_type": listing.get("property_type", "Unknown")
                }

                # Clean metadata to avoid NoneType errors
                cleaned_metadata = clean_metadata(metadata)

                collection.add(
                    documents=[listing["description"]],
                    metadatas=[cleaned_metadata],
                    ids=[listing["id"]],
                )
                current_count = len(collection.get(include=["documents"])['documents'])
                logger.info(f"[store_listings_in_chromadb] Current collection size after adding listing {idx}: {current_count} documents.")
                log_collection_info(collection)
            except Exception as store_error:
                logger.error(f"[store_listings_in_chromadb] Error storing listing {idx}: {store_error}")
                logger.debug(f"[store_listings_in_chromadb] Traceback for listing {idx}: {traceback.format_exc()}")
                continue

        # Final check: log collection size after insertion
        try:
            docs = collection.get(include=["documents", "metadatas"])
            logger.info(f"[store_listings_in_chromadb] Final collection size after storing listings: {len(docs['documents'])} documents.")
            logger.debug(f"[store_listings_in_chromadb] Collection metadata after storing:\n{docs['metadatas']}")
            if len(docs["documents"]) == 0:
                logger.error("[store_listings_in_chromadb] The collection is empty after adding listings. Please verify data flow.")
        except Exception as e:
            logger.warning(f"[store_listings_in_chromadb] Unable to get collection size: {e}")

        logger.info(f"[store_listings_in_chromadb] All {len(listings)} listings have been processed and stored (or skipped) in ChromaDB.")

        # Validate collection embedding dimensions after storing
        validate_collection_after_store(collection)

    except Exception as e:
        logger.error(f"[store_listings_in_chromadb] Error storing listings in ChromaDB: {e}")
        logger.debug(f"[store_listings_in_chromadb] Traceback: {traceback.format_exc()}")

    logger.info("[store_listings_in_chromadb] Step 3 completed successfully.")

def validate_collection_after_store(collection):
    """
    Validate the embeddings in the collection after storing.
    Args:
        collection (Collection): The ChromaDB collection to validate.
    """
    try:
        sample_data = collection.get(include=["embeddings"])
        for idx, embedding in enumerate(sample_data.get("embeddings", [])):
            logger.info(f"[validate_collection_after_store] Validating embedding for stored document {idx}")
            if len(embedding) != EMBEDDING_DIMENSION:
                logger.error(f"[validate_collection_after_store] Embedding mismatch in stored document {idx}. "
                             f"Expected: {EMBEDDING_DIMENSION}, Got: {len(embedding)}")
    except Exception as e:
        logger.error(f"[validate_collection_after_store] Error validating collection embeddings post-store: {e}")


Step 4: Building the User Preference Interface

Collect buyer preferences, such as the number of bedrooms, bathrooms, location, and other specific requirements from a set of questions or telling the buyer to enter their preferences in natural language. You can hard-code the buyer preferences in questions and answers, or collect them interactively however you'd like, example:
questions = [   
                "How big do you want your house to be?" 
                "What are 3 most important things for you in choosing this property?", 
                "Which amenities would you like?", 
                "Which transportation options are important to you?",
                "How urban do you want your neighborhood to be?",   
            ]
answers = [
    "A comfortable three-bedroom house with a spacious kitchen and a cozy living room.",
    "A quiet neighborhood, good local schools, and convenient shopping options.",
    "A backyard for gardening, a two-car garage, and a modern, energy-efficient heating system.",
    "Easy access to a reliable bus line, proximity to a major highway, and bike-friendly roads.",
    "A balance between suburban tranquility and access to urban amenities like restaurants and theaters."
Buyer Preference Parsing: Implement logic to interpret and structure these preferences for querying the vector database.

In [15]:
# Step 4: Parsing Buyer Preferences
import json
import logging
import openai
import re

# Logging setup
logging.basicConfig(level=logging.DEBUG)
logger = logging.getLogger("HomeMatch-Step4")

def parse_json_response(content: str):
    """Attempt to parse a string response as JSON with detailed debugging."""
    logger.info("[parse_json_response] Attempting to parse GPT response as JSON...")
    try:
        parsed_response = json.loads(content)
        logger.debug(f"[parse_json_response] Parsed JSON Response: {json.dumps(parsed_response, indent=4)}")
        return parsed_response
    except json.JSONDecodeError as jde:
        logger.error(f"[parse_json_response] JSON decoding failed: {jde}")
        logger.debug(f"[parse_json_response] Raw response causing JSON error: {content[:200]}...")
        raise ValueError("Non-JSON response from GPT")

def fallback_text_parser(raw_response: str):
    """Parse preferences from unstructured text response."""
    logger.info("[fallback_text_parser] Executing fallback text parser...")
    preferences = {}
    lines = raw_response.strip().splitlines()
    logger.debug(f"[fallback_text_parser] Split raw response into {len(lines)} lines.")

    for line in lines:
        line = line.strip()
        if not line or ":" not in line:
            logger.debug(f"[fallback_text_parser] Skipping line: '{line}' (empty or no key-value structure)")
            continue

        try:
            key, value = line.split(":", 1)
            key = re.sub(r"^\d+\.\s*|\-\s*", "", key.strip().lower())  # Remove numbering or bullets
            preferences[key] = value.strip()
            logger.debug(f"[fallback_text_parser] Extracted preference: '{key}' -> '{value}'")
        except Exception as parse_error:
            logger.debug(f"[fallback_text_parser] Failed to parse line '{line}': {parse_error}")

    if not preferences:
        logger.error("[fallback_text_parser] Fallback text parser failed to extract preferences.")
        raise ValueError("No valid preferences found in fallback text parsing.")

    logger.info(f"[fallback_text_parser] Fallback parsed preferences: {preferences}")
    return preferences

def llm(prompt: str, enforce_json=False):
    """Send a prompt to OpenAI's GPT API and return the structured response."""
    logger.info("[llm] Sending prompt to GPT API...")
    try:
        response = openai.ChatCompletion.create(
            model="gpt-3.5-turbo",
            messages=[
                {"role": "system", "content": "You are a helpful assistant."},
                {"role": "user", "content": prompt},
            ],
        )
        content = response.choices[0].message["content"]
        logger.debug(f"[llm] Raw GPT Response: {content}")

        # If enforce_json is True, attempt JSON parsing directly
        if enforce_json or content.strip().startswith(("{", "[")):
            return parse_json_response(content)

        logger.warning("[llm] GPT response does not appear to be JSON. Returning raw text.")
        return content
    except Exception as e:
        logger.error(f"[llm] An unexpected error occurred during GPT API call: {e}")
        raise

def parse_preferences(input_text: str):
    """Parse buyer preferences using GPT with robust fallback mechanisms."""
    logger.info(f"[parse_preferences] Received input text for parsing: '{input_text}'")

    if not input_text or not isinstance(input_text, str):
        logger.error(f"[parse_preferences] Invalid input text: {input_text}")
        return {}

    try:
        # Force GPT to respond in JSON format on the first attempt
        prompt = (
            "Respond only in JSON format. Extract preferences like this: "
            '{"Property Type": "House", "Number of Bedrooms": 3, "Neighborhood": "Quiet", "Feature": "Backyard"}.\n'
            f"Input: {input_text}"
        )
        logger.info("[parse_preferences] Parsing buyer preferences using GPT...")
        structured_response = llm(prompt, enforce_json=True)
        logger.info("[parse_preferences] Preferences parsed successfully using GPT.")
        logger.debug(f"[parse_preferences] Structured Preferences: {json.dumps(structured_response, indent=4)}")
        return structured_response
    except ValueError as ve:
        logger.warning(f"[parse_preferences] GPT failed with ValueError: {ve}. Attempting fallback parsing.")
        try:
            raw_response = llm(f"Extract structured preferences from the following input: {input_text}")
            return fallback_text_parser(raw_response)
        except Exception as fallback_error:
            logger.error(f"[parse_preferences] Fallback text parsing failed: {fallback_error}")

    logger.info("[parse_preferences] Falling back to local heuristic parser.")
    return local_preference_parser(input_text)

def local_preference_parser(input_text: str):
    """Fallback to simple heuristics to extract preferences."""
    logger.info("[local_preference_parser] Parsing buyer preferences locally using heuristics...")
    preferences = {"bedrooms": None, "neighborhood": None, "features": []}
    words = input_text.lower().split()
    preferences["bedrooms"] = next((int(word) for word in words if word.isdigit()), None)
    if "quiet" in input_text.lower():
        preferences["neighborhood"] = "quiet"
    if "backyard" in input_text.lower() or "garden" in input_text.lower():
        preferences["features"].append("backyard")
    logger.debug(f"[local_preference_parser] Heuristically parsed preferences: {preferences}")
    return preferences

def preferences_to_query(preferences: dict) -> str:
    """
    Convert preferences dictionary into a structured, consistent query string for semantic search.

    Args:
        preferences (dict): Dictionary of buyer preferences.

    Returns:
        str: Generated query string.
    """
    logger.debug(f"[preferences_to_query] Converting preferences to query: {preferences}")

    if not preferences or not isinstance(preferences, dict):
        logger.error("[preferences_to_query] Invalid or empty preferences provided. Cannot generate query.")
        return ""

    # Standardize the structure of the query string
    query_parts = [
        f"{preferences.get('Property Type', 'home')}",
        f"{preferences.get('Number of Bedrooms', 0)} bedrooms",
        f"{preferences.get('Neighborhood', '').strip()}",
        f"with {preferences.get('Feature', '').strip()}"
    ]

    # Remove any empty parts to prevent awkward phrasing
    query_parts = [part for part in query_parts if part.strip()]

    # Concatenate the structured parts
    query_string = ", ".join(query_parts)
    logger.info(f"[preferences_to_query] Generated standardized Query String: '{query_string}'")

    if not query_string:
        logger.error("[preferences_to_query] Generated query string is empty after processing. Aborting search.")
        return ""

    return query_string



Step 5: Searching Based on Preferences

Semantic Search Implementation: Use the structured buyer preferences to perform a semantic search on the vector database, retrieving listings that most closely match the user's requirements.
Listing Retrieval Logic: Fine-tune the retrieval algorithm to ensure that the most relevant listings are selected based on the semantic closeness to the buyer’s preferences.

In [16]:
# Step 5: Searching Based on Preferences Using ChromaDB
import pandas as pd  # Ensure pandas is available for DataFrame usage
import logging
import traceback

# Logging setup
logger = logging.getLogger("HomeMatch-Step5")
logging.basicConfig(level=logging.DEBUG)


def search_vector_store(preferences, n_results=5):
    """
    Perform semantic search on the ChromaDB vector store based on buyer preferences.

    Args:
        preferences (dict): Dictionary of buyer preferences.
        n_results (int): Number of results to return.

    Returns:
        pd.DataFrame: Matched listings.
    """
    try:
        logger.info("[search_vector_store] Starting search operation in ChromaDB...")
        logger.debug(f"[search_vector_store] Received preferences for search:\n{preferences}")

        # Retrieve or validate the collection
        logger.info("[search_vector_store] Retrieving the ChromaDB collection...")
        collection = get_collection(collection_name="real_estate_listings")

        # Validate collection dimensionality before querying
        if not validate_collection_dimensionality(collection):
            logger.warning("[search_vector_store] Collection dimensionality mismatch detected. Recreating collection...")
            collection = get_collection(collection_name="real_estate_listings")
            logger.info(f"[search_vector_store] Recreated collection '{collection.name}' successfully.")

        # Check if the collection is empty
        collection_size = collection.count()  # Using `count()` to avoid key errors
        logger.info(f"[search_vector_store] Collection size before search: {collection_size} documents.")
        if collection_size == 0:
            logger.error("[search_vector_store] The collection is empty. No listings available for search.")
            return pd.DataFrame()

        logger.info(f"[search_vector_store] Collection contains {collection_size} documents.")

        # Validate preferences
        if not preferences:
            logger.error("[search_vector_store] Preferences are empty. Unable to perform search.")
            return pd.DataFrame()

        # Convert preferences to a consistent query string
        query_string = preferences_to_query(preferences)
        logger.info(f"[search_vector_store] Generated query string from preferences: '{query_string}'")

        if not query_string:
            logger.error("[search_vector_store] Query string is empty after processing preferences. Aborting search.")
            return pd.DataFrame()

        # Generate an embedding for the query string
        try:
            logger.info("[search_vector_store] Generating embedding for query string...")
            query_embedding = generate_embeddings(query_string)
            logger.info(f"[search_vector_store] Generated query embedding length: {len(query_embedding)}")

            if not query_embedding or len(query_embedding) != EMBEDDING_DIMENSION:
                logger.error(f"[search_vector_store] Embedding dimension mismatch. "
                             f"Expected: {EMBEDDING_DIMENSION}, but got: {len(query_embedding)}.")
                return pd.DataFrame()

        except Exception as embedding_error:
            logger.error(f"[search_vector_store] Error generating embedding for query string: {embedding_error}")
            logger.debug(f"[search_vector_store] Traceback:\n{traceback.format_exc()}")
            return pd.DataFrame()

        # Perform the search using ChromaDB
        logger.info("[search_vector_store] Performing search with ChromaDB...")
        try:
            search_results = collection.query(
                query_embeddings=[query_embedding],
                n_results=n_results,
                include=["documents", "metadatas"]
            )
            logger.debug(f"[search_vector_store] Raw search results from ChromaDB:\n{search_results}")

            # Check if the result structure is empty
            if not search_results or not search_results.get("documents"):
                logger.warning("[search_vector_store] Search returned no documents.")
                return pd.DataFrame()

            # Log the number of matched documents
            logger.info(f"[search_vector_store] Number of matched documents: {len(search_results.get('documents', []))}")

        except Exception as query_error:
            logger.error(f"[search_vector_store] ChromaDB query failed: {query_error}")
            logger.debug(f"[search_vector_store] Traceback:\n{traceback.format_exc()}")
            return pd.DataFrame()

        # Convert results to a DataFrame for better readability
        results_df = pd.DataFrame({
            "document": search_results.get("documents", []),
            "metadata": search_results.get("metadatas", [])
        })

        # Validate metadata consistency
        if results_df.empty or results_df["metadata"].isnull().all():
            logger.warning("[search_vector_store] No listings matched the buyer's preferences.")
        else:
            logger.info(f"[search_vector_store] Retrieved {len(results_df)} matching listings from ChromaDB.")
            logger.debug(f"[search_vector_store] Matched Listings DataFrame (First 3 rows):\n{results_df.head(3)}")

        return results_df

    except Exception as e:
        logger.error(f"[search_vector_store] An unexpected error occurred during ChromaDB search: {e}")
        logger.debug(f"[search_vector_store] Traceback:\n{traceback.format_exc()}")
        return pd.DataFrame()


def preferences_to_query(preferences: dict) -> str:
    """
    Convert preferences dictionary into a query string for semantic search.

    Args:
        preferences (dict): Dictionary of buyer preferences.

    Returns:
        str: Generated query string.
    """
    logger.info("[preferences_to_query] Converting preferences to a query string...")
    logger.debug(f"[preferences_to_query] Received preferences: {preferences}")

    if not preferences or not isinstance(preferences, dict):
        logger.error("[preferences_to_query] Invalid or empty preferences provided. Cannot generate query.")
        return ""

    query_parts = []
    if preferences.get("Property Type"):
        query_parts.append(preferences["Property Type"])
    if preferences.get("Number of Bedrooms"):
        query_parts.append(f"{preferences['Number of Bedrooms']} bedrooms")
    if preferences.get("Neighborhood"):
        query_parts.append(preferences["Neighborhood"])
    if preferences.get("Feature"):
        query_parts.append(preferences["Feature"])

    query_string = ", ".join(query_parts).strip()
    logger.info(f"[preferences_to_query] Generated Query String: '{query_string}'")

    if not query_string:
        logger.error("[preferences_to_query] Generated query string is empty after processing. Aborting search.")
        return ""

    return query_string



INFO:numexpr.utils:Note: NumExpr detected 16 cores but "NUMEXPR_MAX_THREADS" not set, so enforcing safe limit of 8.
INFO:numexpr.utils:NumExpr defaulting to 8 threads.


Step 6: Personalizing Listing Descriptions

LLM Augmentation: For each retrieved listing, use the LLM to augment the description, tailoring it to resonate with the buyer’s specific preferences. This involves subtly emphasizing aspects of the property that align with what the buyer is looking for.
Maintaining Factual Integrity: Ensure that the augmentation process enhances the appeal of the listing without altering factual information.

In [17]:
# Step 6: Personalizing Listings
import logging
import json

# Logging setup
logger = logging.getLogger("HomeMatch-Step6")
logging.basicConfig(level=logging.DEBUG)

def validate_listing(listing):
    """
    Validate that a listing contains all the required keys and proper formats.

    Args:
        listing (dict): A listing to validate.

    Returns:
        bool: True if valid, False otherwise.

    Raises:
        ValueError: If the listing is missing required keys or invalid formats.
    """
    logger.info("[validate_listing] Validating listing structure...")
    required_keys = {"id", "description", "features"}
    if not isinstance(listing, dict):
        raise ValueError("[validate_listing] Listing must be a dictionary.")
    
    missing_keys = required_keys - listing.keys()
    if missing_keys:
        raise ValueError(f"[validate_listing] Missing keys {missing_keys} in listing {listing}")

    # Validate types of required fields
    if not isinstance(listing["id"], str):
        raise ValueError("[validate_listing] 'id' must be a string.")
    if not isinstance(listing["description"], str):
        raise ValueError("[validate_listing] 'description' must be a string.")
    if not isinstance(listing["features"], list):
        raise ValueError("[validate_listing] 'features' must be a list.")
    
    logger.info("[validate_listing] Listing validation successful.")
    return True


def reformat_listings_for_personalization(listings):
    """
    Reformat listings retrieved from the search result to match the expected structure.
    
    Args:
        listings (list): List of dictionaries, each containing "document" and "metadata".

    Returns:
        list: Reformatted listings.
    """
    logger.info("[reformat_listings_for_personalization] Reformatting listings for personalization...")
    reformatted_listings = []

    for idx, listing_data in enumerate(listings):
        document = listing_data.get("document", "")
        metadata = listing_data.get("metadata", {})

        # Handle cases where document is a list instead of a string
        if isinstance(document, list):
            logger.warning(f"[reformat_listings_for_personalization] Document {idx} is a list. Concatenating...")
            document = " ".join(document)  # Join list of strings into a single string

        # Handle cases where metadata is a list (select the first dictionary if present)
        if isinstance(metadata, list):
            if len(metadata) > 0 and isinstance(metadata[0], dict):
                metadata = metadata[0]  # Take the first dictionary from the list
            else:
                logger.warning(f"[reformat_listings_for_personalization] Metadata {idx} is invalid or empty. Skipping...")
                continue

        # Validate document and metadata types
        if not isinstance(document, str):
            logger.warning(f"[reformat_listings_for_personalization] Document {idx} is not a string after concatenation. Skipping...")
            continue
        if not isinstance(metadata, dict):
            logger.warning(f"[reformat_listings_for_personalization] Metadata {idx} is not a dictionary. Skipping...")
            continue

        # Add to reformatted listings
        reformatted_listings.append({
            "id": metadata.get("id", f"unknown_{idx}"),
            "description": document,
            "features": [],  # Placeholder for feature extraction if needed
            "neighborhood": metadata.get("neighborhood", ""),
            "price": metadata.get("price", 0),
            "bedrooms": metadata.get("bedrooms", 0),
            "property_type": metadata.get("property_type", "Unknown")
        })

    logger.info(f"[reformat_listings_for_personalization] Reformatted {len(reformatted_listings)} listings.")
    return reformatted_listings




def personalize_listing(listing, preferences):
    """
    Personalize a single listing based on buyer preferences.

    Args:
        listing (dict): The listing to personalize.
        preferences (dict): Buyer preferences.

    Returns:
        dict: A personalized listing.

    Raises:
        ValueError: If listing or preferences are invalid.
    """
    logger.info(f"[personalize_listing] Personalizing listing ID: {listing.get('id', 'unknown')}...")
    validate_listing(listing)

    # Extract relevant features for personalization
    description = listing.get("description", "")
    features = listing.get("features", [])
    logger.debug(f"[personalize_listing] Original features: {features}")

    # Adjust description to emphasize preferred features
    emphasized_features = [feature for feature in preferences.values() if feature in features]
    logger.debug(f"[personalize_listing] Emphasized features based on preferences: {emphasized_features}")
    
    personalized_description = description
    if emphasized_features:
        personalized_description += f" This property includes: {', '.join(emphasized_features)}."

    logger.info(f"[personalize_listing] Personalized listing created for ID: {listing['id']}.")

    # Return personalized listing
    return {
        "id": listing["id"],
        "personalized_description": personalized_description,
        "original_description": description,
        "emphasized_features": emphasized_features,
    }

def personalize_all_listings(listings, preferences):
    """
    Personalize a list of listings based on buyer preferences.

    Args:
        listings (dict): A dictionary containing documents and metadata from the vector store.
        preferences (dict): Buyer preferences.

    Returns:
        list: A list of personalized listings.
    """
    logger.info("[personalize_all_listings] Personalizing all listings...")
    
    # Reformat listings to the expected structure
    reformatted_listings = reformat_listings_for_personalization(listings)
    
    personalized_listings = []
    for idx, listing in enumerate(reformatted_listings, start=1):
        try:
            logger.info(f"[personalize_all_listings] Processing listing {idx}: ID = {listing.get('id', 'unknown')}")
            personalized_listing = personalize_listing(listing, preferences)
            logger.debug(f"[personalize_all_listings] Personalized listing {idx}: {json.dumps(personalized_listing, indent=4)}")
            personalized_listings.append(personalized_listing)
        except Exception as e:
            logger.error(f"[personalize_all_listings] Failed to personalize listing {idx}: {e}")

    if not personalized_listings:
        logger.warning("[personalize_all_listings] No listings were personalized successfully.")
    else:
        logger.info(f"[personalize_all_listings] Personalized {len(personalized_listings)} out of {len(reformatted_listings)} listings.")

    return personalized_listings


Step 7: Deliverables and Testing

Test your "HomeMatch" application and make sure it meets all of the requirements in the rubric(opens in a new tab). Your project code will be run when it's assessed. Enter different "buyer preferences" and ensure it works.
Jupyter Notebook/Python Program: Compile the application code in a Jupyter notebook or a standalone Python program. Ensure the code is well-commented and logically structured.
Example Outputs: Include example outputs showcasing how user preferences are processed and how the application generates personalized listing descriptions. You can include these in comments in your application or in a Jupyter notebook that's saved with outputs.

In [18]:
# Step 7: Test the Application with ChromaDB (Full Workflow)
# Import required modules
import traceback  # Handles error stack trace logging
import numpy as np
import chromadb  # Import ChromaDB client
from chromadb.config import Settings
import os
import json
import logging

# Set up logging
logger = logging.getLogger("HomeMatch-Step7")
logging.basicConfig(level=logging.DEBUG)  # Enable DEBUG logging for detailed insights

def test_application_chromadb():
    """
    Full test of the HomeMatch application with ChromaDB by executing Steps 2 to 6 sequentially.
    """
    try:
        logger.info("[test_application_chromadb] ===== Starting Application Test (ChromaDB) =====")
        logger.info("---- Step 2: Generate or Read Listings ---")
        # ---- Step 2: Generate or Read Listings ----
        listings_file_path = "listings.txt"
        if os.path.exists(listings_file_path):
            logger.info("[test_application_chromadb] Reading existing listings from file...")
            generated_listings = read_listings_from_file(listings_file_path)
            if len(generated_listings) == 0:
                logger.warning("[test_application_chromadb] File was read but contained no listings. Generating new listings...")
                generated_listings = generate_real_estate_listings(num_listings=10)
                save_listings_to_file(generated_listings, listings_file_path)
        else:
            logger.info("[test_application_chromadb] No listings file found. Generating new listings...")
            generated_listings = generate_real_estate_listings(num_listings=10)
            save_listings_to_file(generated_listings, listings_file_path)

        # Print listings for verification
        logger.info("[test_application_chromadb] Printing generated listings...")
#        for i, listing in enumerate(generated_listings, start=1):
#            logger.info(f"[test_application_chromadb] Listing {i}:\n{json.dumps(listing, indent=4)}")
            
        
        # ---- Step 3: Store Listings in ChromaDB ----
        logger.info("[test_application_chromadb] Executing Step 3: Storing listings in ChromaDB...")
        logger.info("---- Step 3: Store Listings in ChromaDB ----")
        try:
            collection = get_collection(collection_name="real_estate_listings")
            logger.info(f"[test_application_chromadb] Embedding function in use: {collection._embedding_function}")
            store_listings_in_chromadb(generated_listings)
            logger.info("[test_application_chromadb] Listings stored successfully in ChromaDB.")
            validate_collection_dimensionality(collection)
        except Exception as e:
            logger.error(f"[test_application_chromadb] Step 3 failed with unexpected error: {e}")
            logger.debug(f"[test_application_chromadb] Traceback:\n{traceback.format_exc()}")
  
        # ---- Step 4: Parse Buyer Preferences ----
        logger.info("[test_application_chromadb] Executing Step 4: Parsing Buyer Preferences...")
        logger.info("---- Step 4: Parse Buyer Preferences ----")
        buyer_input = "I want a 3-bedroom house in a quiet neighborhood with a backyard."
        parsed_preferences = parse_preferences(buyer_input)
        logger.info(f"[test_application_chromadb] Parsed Buyer Preferences: {json.dumps(parsed_preferences, indent=4)}")
        query_string = preferences_to_query(parsed_preferences)
        logger.info(f"[test_application_chromadb] Generated Query String: '{query_string}'")
 
        # ---- Step 5: Semantic Search in ChromaDB ----
        logger.info("[test_application_chromadb] Executing Step 5: Searching vector database...")
        logger.info("---- Step 5: Semantic Search in ChromaDB ----")
        try:
            matched_listings_df = search_vector_store(parsed_preferences)  # Pass parsed preferences
            if not matched_listings_df.empty:
                logger.info(f"[test_application_chromadb] Matched Listings:\n{matched_listings_df}")
            else:
                logger.warning("[test_application_chromadb] No listings matched the buyer's preferences.")
                return  # Stop further processing if no matches

            # Convert DataFrame to a list of dictionaries for Step 6
            matched_listings = matched_listings_df.to_dict(orient="records")
        except Exception as e:
            logger.error(f"[test_application_chromadb] Step 5 failed with unexpected error: {e}")
            logger.debug(f"[test_application_chromadb] Traceback:\n{traceback.format_exc()}")
            return

        # ---- Step 6: Personalize Listings ----
        logger.info("[test_application_chromadb] Executing Step 6: Personalizing listings...")
        logger.info("---- Step 6: Personalize Listings ----")
        try:
            personalized_listings = personalize_all_listings(matched_listings, parsed_preferences)
            if personalized_listings:
                logger.info("[test_application_chromadb] Personalization Complete.")
                logger.debug(f"[test_application_chromadb] Personalized listings:\n{json.dumps(personalized_listings, indent=4)}")
            else:
                logger.warning("[test_application_chromadb] No listings were personalized successfully.")
        except Exception as e:
            logger.error(f"[test_application_chromadb] Step 6 failed with unexpected error: {e}")
            logger.debug(f"[test_application_chromadb] Traceback:\n{traceback.format_exc()}")

        
        
    except Exception as e:
        logger.error(f"[test_application_chromadb] Test failed with unexpected error: {e}")
        logger.debug(f"[test_application_chromadb] Traceback:\n{traceback.format_exc()}")

# Run the test
logger.info("[Main] ===== Running Application Test (ChromaDB) =====")
test_application_chromadb()


INFO:HomeMatch-Step7:[Main] ===== Running Application Test (ChromaDB) =====
INFO:HomeMatch-Step7:[test_application_chromadb] ===== Starting Application Test (ChromaDB) =====
INFO:HomeMatch-Step7:---- Step 2: Generate or Read Listings ---
INFO:HomeMatch-Step7:[test_application_chromadb] Reading existing listings from file...
INFO:HomeMatch-Step7:[read_listings_from_file] Reading listings from file...
ERROR:HomeMatch-Step7:[read_listings_from_file] Error parsing JSON from listings.txt: Expecting value: line 1 column 1 (char 0). Deleting corrupted file.
INFO:HomeMatch-Step7:[generate_real_estate_listings] Generating 10 real estate listings using OpenAI GPT...
DEBUG:openai:message='Request to OpenAI API' method=post path=https://api.openai.com/v1/chat/completions
DEBUG:openai:api_version=None data='{"model": "gpt-3.5-turbo", "messages": [{"role": "system", "content": "You are a helpful assistant."}, {"role": "user", "content": "You are a real estate expert. Generate a realistic real estate

DEBUG:urllib3.connectionpool:https://api.openai.com:443 "POST /v1/chat/completions HTTP/1.1" 200 None
DEBUG:openai:message='OpenAI API response' path=https://api.openai.com/v1/chat/completions processing_ms=2025 request_id=req_7e03d4aa08789a7bd832fa7b761c52d4 response_code=200
DEBUG:HomeMatch-Step7:[generate_real_estate_listings] Generated Listing Text: Bedrooms: 4
Bathrooms: 3.5
House Size: 2,800 sqft
Description: This stunning 4-bedroom, 3.5-bathroom home in the desirable Green Oaks neighborhood is a true gem. The property features a spacious open-concept layout, a modern kitchen with granite countertops and stainless steel appliances, a cozy fireplace in the living room, and a luxurious master suite with a spa-like bathroom. The backyard is perfect for entertaining with a large deck and beautifully landscaped yard. Additional highlights include a home office, hardwood floors throughout, and a 2-car garage.

Neighborhood Description: Green Oaks is known for its peaceful, family-frien

DEBUG:openai:message='Request to OpenAI API' method=post path=https://api.openai.com/v1/chat/completions
DEBUG:openai:api_version=None data='{"model": "gpt-3.5-turbo", "messages": [{"role": "system", "content": "You are a helpful assistant."}, {"role": "user", "content": "You are a real estate expert. Generate a realistic real estate listing with the following format:\\nNeighborhood: <neighborhood name>\\nPrice: $<price>\\nBedrooms: <number of bedrooms>\\nBathrooms: <number of bathrooms>\\nHouse Size: <square footage>\\nDescription: <detailed description of the property, including features>\\nNeighborhood Description: <brief description of the neighborhood and amenities nearby>\\nNeighborhood: Willow Creek\\nPrice: $634617\\n"}], "max_tokens": 300, "temperature": 0.7}' message='Post details'
DEBUG:urllib3.connectionpool:https://api.openai.com:443 "POST /v1/chat/completions HTTP/1.1" 200 None
DEBUG:openai:message='OpenAI API response' path=https://api.openai.com/v1/chat/completions proc

INFO:HomeMatch-Step7:[generate_real_estate_listings] Generated Listing 8: {
    "id": "listing_8",
    "neighborhood": "Brookside",
    "price": 643919,
    "description": "I can certainly help with that. Here's a realistic real estate listing for you:\n\nNeighborhood: Brookside\nPrice: $643,919\nBedrooms: 3\nBathrooms: 2.5\nHouse Size: 2,100 sqft\nDescription: Welcome to this charming 3-bedroom, 2.5-bathroom home located in the desirable Brookside neighborhood. This well-maintained property features a spacious living room with a cozy fireplace, a modern kitchen with stainless steel appliances, and a master suite with a walk-in closet. The backyard offers a peaceful retreat with a patio and lush landscaping, perfect for outdoor entertaining. Additional highlights include a two-car garage and plenty of storage space.\n\nNeighborhood Description: Brookside is known for its tree-lined streets, friendly community vibe, and convenient access to parks, schools, and shopping centers. Resident

INFO:HomeMatch-Step7:[validate_listings] Validating Listing 2: {
    "id": "listing_2",
    "neighborhood": "Willow Creek",
    "price": 816852,
    "description": "Neighborhood: Willow Creek\nPrice: $816,852\nBedrooms: 4\nBathrooms: 3\nHouse Size: 2,500 sqft\n\nDescription: Welcome to this charming 4-bedroom, 3-bathroom home located in the desirable Willow Creek neighborhood. This well-maintained property features a spacious living room with a cozy fireplace, a modern kitchen with stainless steel appliances and granite countertops, a formal dining room for entertaining guests, and a master suite with a luxurious en-suite bathroom. The property also boasts a beautifully landscaped backyard with a patio area perfect for outdoor gatherings.\n\nNeighborhood Description: Willow Creek is known for its family-friendly atmosphere, top-rated schools, and convenient access to parks, shopping centers, and dining options. Residents enjoy a peaceful suburban lifestyle while still being close to ci

...

INFO:HomeMatch-Step7:[store_listings_in_chromadb] Generated embedding for listing 8 (length): 1536
DEBUG:openai:message='Request to OpenAI API' method=post path=https://api.openai.com/v1/engines/text-embedding-ada-002/embeddings
DEBUG:openai:api_version=None data='{"input": ["I can certainly help with that. Here\'s a realistic real estate listing for you:  Neighborhood: Brookside Price: $643,919 Bedrooms: 3 Bathrooms: 2.5 House Size: 2,100 sqft Description: Welcome to this charming 3-bedroom, 2.5-bathroom home located in the desirable Brookside neighborhood. This well-maintained property features a spacious living room with a cozy fireplace, a modern kitchen with stainless steel appliances, and a master suite with a walk-in closet. The backyard offers a peaceful retreat with a patio and lush landscaping, perfect for outdoor entertaining. Additional highlights include a two-car garage and plenty of storage space.  Neighborhood Description: Brookside is known for its tree-lined streets, 

INFO:HomeMatch-Step7:[log_collection_info] Collection 'real_estate_listings' contains 9 documents.
INFO:HomeMatch-Step7:[log_collection_info] Sample 1: Document = I'm glad to help with that! Here's a realistic real estate listing based on the information you prov..., Metadata = {'bedrooms': 0, 'id': 'listing_1', 'neighborhood': 'Brookside', 'price': 369539, 'property_type': 'Unknown'}
INFO:HomeMatch-Step7:[log_collection_info] Sample 2: Document = Neighborhood: Willow Creek
Price: $816,852
Bedrooms: 4
Bathrooms: 3
House Size: 2,500 sqft

Descript..., Metadata = {'bedrooms': 0, 'id': 'listing_2', 'neighborhood': 'Willow Creek', 'price': 816852, 'property_type': 'Unknown'}
INFO:HomeMatch-Step7:[log_collection_info] Sample 3: Document = Bedrooms: 4
Bathrooms: 3.5
House Size: 2,800 sqft
Description: This stunning 4-bedroom, 3.5-bathroom..., Metadata = {'bedrooms': 0, 'id': 'listing_3', 'neighborhood': 'Green Oaks', 'price': 722702, 'property_type': 'Unknown'}
INFO:HomeMatch-Step7:[store_li

INFO:HomeMatch-Step7:[store_listings_in_chromadb] All 10 listings have been processed and stored (or skipped) in ChromaDB.
INFO:HomeMatch-Step7:[validate_collection_after_store] Validating embedding for stored document 0
INFO:HomeMatch-Step7:[validate_collection_after_store] Validating embedding for stored document 1
INFO:HomeMatch-Step7:[validate_collection_after_store] Validating embedding for stored document 2
INFO:HomeMatch-Step7:[validate_collection_after_store] Validating embedding for stored document 3
INFO:HomeMatch-Step7:[validate_collection_after_store] Validating embedding for stored document 4
INFO:HomeMatch-Step7:[validate_collection_after_store] Validating embedding for stored document 5
INFO:HomeMatch-Step7:[validate_collection_after_store] Validating embedding for stored document 6
INFO:HomeMatch-Step7:[validate_collection_after_store] Validating embedding for stored document 7
INFO:HomeMatch-Step7:[validate_collection_after_store] Validating embedding for stored docume

INFO:HomeMatch-Step7:[search_vector_store] Number of matched documents: 1
INFO:HomeMatch-Step7:[search_vector_store] Retrieved 1 matching listings from ChromaDB.
DEBUG:HomeMatch-Step7:[search_vector_store] Matched Listings DataFrame (First 3 rows):
                                            document  \
0  [Bedrooms: 4\nBathrooms: 3.5\nHouse Size: 2,80...   

                                            metadata  
0  [{'bedrooms': 0, 'id': 'listing_3', 'neighborh...  
INFO:HomeMatch-Step7:[test_application_chromadb] Matched Listings:
                                            document  \
0  [Bedrooms: 4\nBathrooms: 3.5\nHouse Size: 2,80...   

                                            metadata  
0  [{'bedrooms': 0, 'id': 'listing_3', 'neighborh...  
INFO:HomeMatch-Step7:[test_application_chromadb] Executing Step 6: Personalizing listings...
INFO:HomeMatch-Step7:---- Step 6: Personalize Listings ----
INFO:HomeMatch-Step7:[personalize_all_listings] Personalizing all listings...
INFO:Ho

INFO:HomeMatch-Step7:[personalize_all_listings] Personalized 1 out of 1 listings.
INFO:HomeMatch-Step7:[test_application_chromadb] Personalization Complete.
DEBUG:HomeMatch-Step7:[test_application_chromadb] Personalized listings:
[
    {
        "id": "listing_3",
        "personalized_description": "Bedrooms: 4\nBathrooms: 3.5\nHouse Size: 2,800 sqft\nDescription: This stunning 4-bedroom, 3.5-bathroom home in the desirable Green Oaks neighborhood is a true gem. The property features a spacious open-concept layout, a modern kitchen with granite countertops and stainless steel appliances, a cozy fireplace in the living room, and a luxurious master suite with a spa-like bathroom. The backyard is perfect for entertaining with a large deck and beautifully landscaped yard. Additional highlights include a home office, hardwood floors throughout, and a 2-car garage.\n\nNeighborhood Description: Green Oaks is known for its peaceful, family-friendly atmosphere and tree-lined streets. Residents 

In [19]:
logger.info("Finished...")

INFO:HomeMatch-Step7:Finished...
DEBUG:urllib3.connectionpool:https://us-api.i.posthog.com:443 "POST /batch/ HTTP/1.1" 200 15
